In [35]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Name: Shrujal                        
R.No. : 101903426                            
Group : 3CO16

In [36]:
import numpy as np
import pandas as pd

import cudf as cd
import cuml as ml
import cupy as cp

import sklearn

 # Reading the csv file and converting into dataframe

In [37]:
df = cd.read_csv('/kaggle/input/110-1-ntut-dl-app-hw3/IXIC.csv')
data = cd.DataFrame(data = df,columns=['Date','Open','High','Low','Close','Adj Close'])
data['target']=df.iloc[:,-1];
data

In [38]:
data.isnull().sum()
type(data)

# Removing the date feature

In [39]:
df = data.drop(['Date'],axis = 1)

In [40]:
df

# Determining the size of the dataset

In [41]:
df.shape

# Splitting the data in X and Y 

In [42]:
X = df.iloc[:,:-1]
Y = df.iloc[:,-1]
Y = cp.asarray(Y)
Y=Y.reshape(-1,1)

# Performing Standardization on the Data

In [43]:
from cuml.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled

# Splitting the data for training and testing

In [44]:
from cuml.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X_scaled,Y,test_size=0.3,random_state = 42)

In [45]:
print(X_train.shape)
print(X_test.shape)

print(Y_train.shape)
print(Y_test.shape)
y_test = Y_test.astype('float64')

# Performing Linear Regression and analyzing evaluation metrics on the dataset

In [46]:
from cuml.linear_model import LinearRegression

In [47]:
lin_reg = LinearRegression(fit_intercept = True, normalize = False, algorithm = 'svd')
reg = lin_reg.fit(X_train,Y_train)
print("Coefficients:")
print(reg.coef_)
print("Intercept:")
print(reg.intercept_)

predict = lin_reg.predict(X_test)
print("Predictions:")
print(predict)

print("Mean Squared Error:")
print(ml.metrics.regression.mean_squared_error(y_test,predict))

print("R2 Score:")
print(ml.metrics.regression.r2_score(y_test,predict))

print("Mean Absolute Error:")
print(ml.metrics.regression.mean_absolute_error(y_test,predict))